Analyze Herd Dynamics Using Machine Learning and Simplified Functions

In [18]:
# Machine Learning Herd Manager
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple, Dict
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import pandas as pd

@dataclass
class Prey:
    species: str
    speed: float  # meters per second
    weight: float  # kg
    herd_size: int
    daily_arrival_rate: float  # average number of herds per day

@dataclass
class HuntingZone:
    name: str
    distance_from_pride: float  # meters
    water_proximity: float  # meters to nearest water source
    cover_density: float  # 0-1 scale of hiding spots

class MLPrideResourceManager:
    def __init__(self, hunting_zones: List[HuntingZone], prey_types: List[Prey], 
                 pride_size: int = 6, food_per_lion: float = 5.0):
        self.hunting_zones = hunting_zones
        self.prey_types = prey_types
        self.pride_size = pride_size
        self.food_per_lion = food_per_lion
        self.daily_food_requirement = food_per_lion * pride_size
        self.ml_model = None
        self.scaler = StandardScaler()
        
    def generate_training_data(self, num_samples: int = 1000) -> Tuple[np.ndarray, np.ndarray]:
        """Generate synthetic data for training the ML model."""
        data = []
        targets = []
        
        for _ in range(num_samples):
            for zone in self.hunting_zones:
                for prey in self.prey_types:
                    features = [
                        prey.speed,
                        prey.weight,
                        prey.herd_size,
                        prey.daily_arrival_rate,
                        zone.distance_from_pride,
                        zone.water_proximity,
                        zone.cover_density,
                        self.pride_size,
                        self.daily_food_requirement
                    ]
                    
                    # Add some random variation
                    features = np.array(features) * np.random.normal(1, 0.1, len(features))
                    
                    # Calculate synthetic success probability
                    base_success = zone.cover_density * 0.7
                    speed_factor = 1 - (prey.speed / 100)
                    distance_penalty = 1 - (zone.distance_from_pride / 5000)
                    water_bonus = 1 - (zone.water_proximity / 1000)
                    
                    success_prob = (base_success * speed_factor * distance_penalty * 
                                  (1 + 0.2 * water_bonus))
                    success_prob = min(max(success_prob, 0.1), 0.9)
                    
                    data.append(features)
                    targets.append(success_prob)
        
        return np.array(data), np.array(targets)

    def train_ml_model(self):
        """Train the ML model on synthetic data."""
        X, y = self.generate_training_data()
        X_scaled = self.scaler.fit_transform(X)
        
        self.ml_model = RandomForestRegressor(n_estimators=100, random_state=42)
        self.ml_model.fit(X_scaled, y)
        
    def predict_success_probability(self, prey: Prey, zone: HuntingZone) -> float:
        """Use ML model to predict hunt success probability."""
        if self.ml_model is None:
            self.train_ml_model()
            
        features = np.array([[
            prey.speed,
            prey.weight,
            prey.herd_size,
            prey.daily_arrival_rate,
            zone.distance_from_pride,
            zone.water_proximity,
            zone.cover_density,
            self.pride_size,
            self.daily_food_requirement
        ]])
        
        features_scaled = self.scaler.transform(features)
        return float(self.ml_model.predict(features_scaled)[0])

    def simulate_daily_outcomes(self, days: int = 30) -> Tuple[float, float, Dict]:
        """Simulate resource acquisition using ML predictions."""
        if self.ml_model is None:
            self.train_ml_model()
            
        total_food_acquired = 0
        successful_hunts = 0
        total_hunts = 0
        
        zone_stats = {zone.name: {'attempts': 0, 'successes': 0} for zone in self.hunting_zones}
        prey_stats = {prey.species: {'attempts': 0, 'successes': 0} for prey in self.prey_types}
        
        for day in range(days):
            daily_food = 0
            print(f"\nDay {day + 1}:")
            
            for zone in self.hunting_zones:
                for prey in self.prey_types:
                    daily_herds = np.random.poisson(prey.daily_arrival_rate)
                    
                    if daily_herds > 0:
                        print(f"  {daily_herds} {prey.species} herds spotted in {zone.name}")
                    
                    for _ in range(daily_herds):
                        total_hunts += 1
                        zone_stats[zone.name]['attempts'] += 1
                        prey_stats[prey.species]['attempts'] += 1
                        
                        success_prob = self.predict_success_probability(prey, zone)
                        
                        if random.random() < success_prob:
                            successful_hunts += 1
                            zone_stats[zone.name]['successes'] += 1
                            prey_stats[prey.species]['successes'] += 1
                            daily_food += prey.weight
                            print(f"    Successful hunt! Caught {prey.species} (+{prey.weight}kg)")
                            
                            if daily_food >= self.daily_food_requirement:
                                print(f"    Daily requirement met: {daily_food:.1f}kg")
                                break
                    
                    if daily_food >= self.daily_food_requirement:
                        break
            
            if daily_food < self.daily_food_requirement:
                print(f"    Day ended with insufficient food: {daily_food:.1f}kg / {self.daily_food_requirement}kg")
                
            total_food_acquired += daily_food
            
        avg_daily_food = total_food_acquired / days
        hunt_success_rate = successful_hunts / total_hunts if total_hunts > 0 else 0
        
        return avg_daily_food, hunt_success_rate, {'zones': zone_stats, 'prey': prey_stats}

def run_simulation(pride_size: int = 6, food_per_lion: float = 5.0, days: int = 30,
                  custom_prey: List[Dict] = None, custom_zones: List[Dict] = None):
    """Run simulation with user-defined parameters."""
    
    # Use default or custom prey types
    if custom_prey is None:
        prey_types = [
            Prey("Gazelle", speed=80, weight=50, herd_size=20, daily_arrival_rate=2.5),
            Prey("Wildebeest", speed=60, weight=200, herd_size=50, daily_arrival_rate=0.8),
            Prey("Zebra", speed=65, weight=350, herd_size=30, daily_arrival_rate=1.2)
        ]
    else:
        prey_types = [Prey(**p) for p in custom_prey]
    
    # Use default or custom hunting zones
    if custom_zones is None:
        hunting_zones = [
            HuntingZone("Riverside", distance_from_pride=800, water_proximity=100, cover_density=0.7),
            HuntingZone("Savanna", distance_from_pride=2000, water_proximity=500, cover_density=0.3),
            HuntingZone("Woodland", distance_from_pride=1500, water_proximity=300, cover_density=0.8)
        ]
    else:
        hunting_zones = [HuntingZone(**z) for z in custom_zones]

    # Initialize and run simulation
    print("\nStarting Pride Resource Management Simulation...")
    print(f"Pride size: {pride_size} lions")
    print(f"Daily food requirement per lion: {food_per_lion}kg")
    print(f"Total daily food requirement: {pride_size * food_per_lion}kg")
    
    print("\nHunting Zones:")
    for zone in hunting_zones:
        print(f"- {zone.name}: {zone.distance_from_pride}m from pride, "
              f"{zone.water_proximity}m from water, {zone.cover_density:.1%} cover density")
    
    print("\nPrey Types:")
    for prey in prey_types:
        print(f"- {prey.species}: {prey.speed}m/s speed, {prey.weight}kg weight, "
              f"herds of {prey.herd_size}, {prey.daily_arrival_rate} herds/day average")

    pride_manager = MLPrideResourceManager(hunting_zones, prey_types, pride_size, food_per_lion)
    avg_food, success_rate, stats = pride_manager.simulate_daily_outcomes(days=days)

    print("\nSimulation Results:")
    print(f"Average daily food acquired: {avg_food:.1f}kg")
    print(f"Overall hunt success rate: {success_rate:.1%}")

    print("\nSuccess rates by hunting zone:")
    for zone_name, zone_data in stats['zones'].items():
        success_rate = zone_data['successes'] / zone_data['attempts'] if zone_data['attempts'] > 0 else 0
        print(f"- {zone_name}: {success_rate:.1%} ({zone_data['successes']}/{zone_data['attempts']} successful hunts)")

    print("\nSuccess rates by prey type:")
    for prey_name, prey_data in stats['prey'].items():
        success_rate = prey_data['successes'] / prey_data['attempts'] if prey_data['attempts'] > 0 else 0
        print(f"- {prey_name}: {success_rate:.1%} ({prey_data['successes']}/{prey_data['attempts']} successful hunts)")

# Example usage with default parameters
run_simulation()

# Example usage with custom parameters
"""
custom_prey = [
    {
        "species": "Antelope",
        "speed": 70,
        "weight": 40,
        "herd_size": 15,
        "daily_arrival_rate": 3.0
    }
]

custom_zones = [
    {
        "name": "Valley",
        "distance_from_pride": 1000,
        "water_proximity": 200,
        "cover_density": 0.6
    }
]

run_simulation(
    pride_size=8,
    food_per_lion=6.0,
    days=15,
    custom_prey=custom_prey,
    custom_zones=custom_zones
)
"""


Starting Pride Resource Management Simulation...
Pride size: 6 lions
Daily food requirement per lion: 5.0kg
Total daily food requirement: 30.0kg

Hunting Zones:
- Riverside: 800m from pride, 100m from water, 70.0% cover density
- Savanna: 2000m from pride, 500m from water, 30.0% cover density
- Woodland: 1500m from pride, 300m from water, 80.0% cover density

Prey Types:
- Gazelle: 80m/s speed, 50kg weight, herds of 20, 2.5 herds/day average
- Wildebeest: 60m/s speed, 200kg weight, herds of 50, 0.8 herds/day average
- Zebra: 65m/s speed, 350kg weight, herds of 30, 1.2 herds/day average

Day 1:
  2 Gazelle herds spotted in Riverside
  2 Gazelle herds spotted in Savanna
  2 Wildebeest herds spotted in Savanna
  1 Zebra herds spotted in Savanna
  2 Gazelle herds spotted in Woodland
  1 Zebra herds spotted in Woodland
    Successful hunt! Caught Zebra (+350kg)
    Daily requirement met: 350.0kg

Day 2:
  2 Gazelle herds spotted in Riverside
  1 Wildebeest herds spotted in Riverside
  1 Ga

'\ncustom_prey = [\n    {\n        "species": "Antelope",\n        "speed": 70,\n        "weight": 40,\n        "herd_size": 15,\n        "daily_arrival_rate": 3.0\n    }\n]\n\ncustom_zones = [\n    {\n        "name": "Valley",\n        "distance_from_pride": 1000,\n        "water_proximity": 200,\n        "cover_density": 0.6\n    }\n]\n\nrun_simulation(\n    pride_size=8,\n    food_per_lion=6.0,\n    days=15,\n    custom_prey=custom_prey,\n    custom_zones=custom_zones\n)\n'

In [9]:
# Backup, non-machine learning-based model

import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
import random

@dataclass
class Prey:
    species: str
    speed: float  # meters per second
    weight: float  # kg
    herd_size: int
    daily_arrival_rate: float  # average number of herds per day
    
@dataclass
class HuntingZone:
    name: str
    distance_from_pride: float  # meters
    water_proximity: float  # meters to nearest water source
    cover_density: float  # 0-1 scale of hiding spots
    
class PrideResourceManager:
    def __init__(self, hunting_zones: List[HuntingZone], prey_types: List[Prey]):
        self.hunting_zones = hunting_zones
        self.prey_types = prey_types
        self.pride_size = 6  # typical pride size
        self.daily_food_requirement = 5 * self.pride_size  # kg per day for whole pride
        
    def calculate_hunt_success_probability(self, prey: Prey, zone: HuntingZone) -> float:
        """Calculate probability of successful hunt based on various factors."""
        # Base success rate influenced by cover density
        base_success_rate = zone.cover_density * 0.7
        
        # Speed factor - higher prey speed reduces success rate
        speed_factor = 1 - (prey.speed / 100)  # normalized to typical prey speeds
        
        # Distance penalty - hunting efficiency decreases with distance from pride
        distance_penalty = 1 - (zone.distance_from_pride / 5000)  # normalized to 5km
        
        # Water proximity bonus - prey are more vulnerable near water
        water_bonus = 1 - (zone.water_proximity / 1000)  # normalized to 1km
        
        success_probability = (base_success_rate * speed_factor * distance_penalty * 
                             (1 + 0.2 * water_bonus))
        
        return min(max(success_probability, 0.1), 0.9)  # bound between 10% and 90%
    
    def simulate_daily_outcomes(self, days: int = 30) -> Tuple[float, float, dict]:
        """Simulate resource acquisition over specified number of days."""
        total_food_acquired = 0
        successful_hunts = 0
        total_hunts = 0
        
        # Track success by zone and prey type
        zone_stats = {zone.name: {'attempts': 0, 'successes': 0} for zone in self.hunting_zones}
        prey_stats = {prey.species: {'attempts': 0, 'successes': 0} for prey in self.prey_types}
        
        for day in range(days):
            daily_food = 0
            print(f"\nDay {day + 1}:")
            
            for zone in self.hunting_zones:
                for prey in self.prey_types:
                    # Simulate prey arrivals based on Poisson distribution
                    daily_herds = np.random.poisson(prey.daily_arrival_rate)
                    
                    if daily_herds > 0:
                        print(f"  {daily_herds} {prey.species} herds spotted in {zone.name}")
                    
                    for _ in range(daily_herds):
                        total_hunts += 1
                        zone_stats[zone.name]['attempts'] += 1
                        prey_stats[prey.species]['attempts'] += 1
                        
                        success_prob = self.calculate_hunt_success_probability(prey, zone)
                        
                        if random.random() < success_prob:
                            successful_hunts += 1
                            zone_stats[zone.name]['successes'] += 1
                            prey_stats[prey.species]['successes'] += 1
                            daily_food += prey.weight
                            print(f"    Successful hunt! Caught {prey.species} (+{prey.weight}kg)")
                            
                            if daily_food >= self.daily_food_requirement:
                                print(f"    Daily requirement met: {daily_food:.1f}kg")
                                break
                    
                    if daily_food >= self.daily_food_requirement:
                        break
            
            if daily_food < self.daily_food_requirement:
                print(f"    Day ended with insufficient food: {daily_food:.1f}kg / {self.daily_food_requirement}kg")
                
            total_food_acquired += daily_food
            
        avg_daily_food = total_food_acquired / days
        hunt_success_rate = successful_hunts / total_hunts if total_hunts > 0 else 0
        
        return avg_daily_food, hunt_success_rate, {'zones': zone_stats, 'prey': prey_stats}

# Create example data
prey_types = [
    Prey("Gazelle", speed=80, weight=50, herd_size=20, daily_arrival_rate=2.5),
    Prey("Wildebeest", speed=60, weight=200, herd_size=50, daily_arrival_rate=0.8),
    Prey("Zebra", speed=65, weight=350, herd_size=30, daily_arrival_rate=1.2)
]

hunting_zones = [
    HuntingZone("Riverside", distance_from_pride=800, water_proximity=100, cover_density=0.7),
    HuntingZone("Savanna", distance_from_pride=2000, water_proximity=500, cover_density=0.3),
    HuntingZone("Woodland", distance_from_pride=1500, water_proximity=300, cover_density=0.8)
]

# Run simulation
print("\nStarting Pride Resource Management Simulation...")
print(f"Pride size: 6 lions")
print(f"Daily food requirement: 30kg")
print("\nHunting Zones:")
for zone in hunting_zones:
    print(f"- {zone.name}: {zone.distance_from_pride}m from pride, {zone.water_proximity}m from water, {zone.cover_density:.1%} cover density")
print("\nPrey Types:")
for prey in prey_types:
    print(f"- {prey.species}: {prey.speed}m/s speed, {prey.weight}kg weight, herds of {prey.herd_size}, {prey.daily_arrival_rate} herds/day average")

pride_manager = PrideResourceManager(hunting_zones, prey_types)
avg_food, success_rate, stats = pride_manager.simulate_daily_outcomes(days=7)  # Reduced to 7 days for clearer output

print("\nSimulation Results:")
print(f"Average daily food acquired: {avg_food:.1f}kg")
print(f"Overall hunt success rate: {success_rate:.1%}")

print("\nSuccess rates by hunting zone:")
for zone_name, zone_data in stats['zones'].items():
    success_rate = zone_data['successes'] / zone_data['attempts'] if zone_data['attempts'] > 0 else 0
    print(f"- {zone_name}: {success_rate:.1%} ({zone_data['successes']}/{zone_data['attempts']} successful hunts)")

print("\nSuccess rates by prey type:")
for prey_name, prey_data in stats['prey'].items():
    success_rate = prey_data['successes'] / prey_data['attempts'] if prey_data['attempts'] > 0 else 0
    print(f"- {prey_name}: {success_rate:.1%} ({prey_data['successes']}/{prey_data['attempts']} successful hunts)")


Starting Pride Resource Management Simulation...
Pride size: 6 lions
Daily food requirement: 30kg

Hunting Zones:
- Riverside: 800m from pride, 100m from water, 70.0% cover density
- Savanna: 2000m from pride, 500m from water, 30.0% cover density
- Woodland: 1500m from pride, 300m from water, 80.0% cover density

Prey Types:
- Gazelle: 80m/s speed, 50kg weight, herds of 20, 2.5 herds/day average
- Wildebeest: 60m/s speed, 200kg weight, herds of 50, 0.8 herds/day average
- Zebra: 65m/s speed, 350kg weight, herds of 30, 1.2 herds/day average

Day 1:
  6 Gazelle herds spotted in Riverside
  1 Wildebeest herds spotted in Riverside
  2 Zebra herds spotted in Riverside
  2 Gazelle herds spotted in Savanna
    Successful hunt! Caught Gazelle (+50kg)
    Daily requirement met: 50.0kg
  3 Gazelle herds spotted in Woodland

Day 2:
  2 Gazelle herds spotted in Riverside
    Successful hunt! Caught Gazelle (+50kg)
    Daily requirement met: 50.0kg
  2 Gazelle herds spotted in Savanna
  3 Gazelle 

In [13]:
#Backup non-machine-learning based script; breaks at day 30.

import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
import random

@dataclass
class Prey:
    species: str
    speed: float  # meters per second
    weight: float  # kg
    herd_size: int
    daily_arrival_rate: float  # average number of herds per day
    
@dataclass
class HuntingZone:
    name: str
    distance_from_pride: float  # meters
    water_proximity: float  # meters to nearest water source
    cover_density: float  # 0-1 scale of hiding spots
    
class PrideResourceManager:
    def __init__(self, hunting_zones: List[HuntingZone], prey_types: List[Prey]):
        self.hunting_zones = hunting_zones
        self.prey_types = prey_types
        self.pride_size = 6  # typical pride size
        self.daily_food_requirement = 5 * self.pride_size  # kg per day for whole pride
        
    def calculate_hunt_success_probability(self, prey: Prey, zone: HuntingZone) -> float:
        """Calculate probability of successful hunt based on various factors."""
        # Base success rate influenced by cover density
        base_success_rate = zone.cover_density * 0.7
        
        # Speed factor - higher prey speed reduces success rate
        speed_factor = 1 - (prey.speed / 100)  # normalized to typical prey speeds
        
        # Distance penalty - hunting efficiency decreases with distance from pride
        distance_penalty = 1 - (zone.distance_from_pride / 5000)  # normalized to 5km
        
        # Water proximity bonus - prey are more vulnerable near water
        water_bonus = 1 - (zone.water_proximity / 1000)  # normalized to 1km
        
        success_probability = (base_success_rate * speed_factor * distance_penalty * 
                             (1 + 0.2 * water_bonus))
        
        return min(max(success_probability, 0.1), 0.9)  # bound between 10% and 90%
    
    def simulate_daily_outcomes(self, days: int = 30) -> Tuple[float, float, dict]:
        """Simulate resource acquisition over specified number of days."""
        total_food_acquired = 0
        successful_hunts = 0
        total_hunts = 0
        
        # Track success by zone and prey type
        zone_stats = {zone.name: {'attempts': 0, 'successes': 0} for zone in self.hunting_zones}
        prey_stats = {prey.species: {'attempts': 0, 'successes': 0} for prey in self.prey_types}
        
        for day in range(days):
            daily_food = 0
            print(f"\nDay {day + 1}:")
            
            for zone in self.hunting_zones:
                for prey in self.prey_types:
                    # Simulate prey arrivals based on Poisson distribution
                    daily_herds = np.random.poisson(prey.daily_arrival_rate)
                    
                    if daily_herds > 0:
                        print(f"  {daily_herds} {prey.species} herds spotted in {zone.name}")
                    
                    for _ in range(daily_herds):
                        total_hunts += 1
                        zone_stats[zone.name]['attempts'] += 1
                        prey_stats[prey.species]['attempts'] += 1
                        
                        success_prob = self.calculate_hunt_success_probability(prey, zone)
                        
                        if random.random() < success_prob:
                            successful_hunts += 1
                            zone_stats[zone.name]['successes'] += 1
                            prey_stats[prey.species]['successes'] += 1
                            daily_food += prey.weight
                            print(f"    Successful hunt! Caught {prey.species} (+{prey.weight}kg)")
                            
                            if daily_food >= self.daily_food_requirement:
                                print(f"    Daily requirement met: {daily_food:.1f}kg")
                                break
                    
                    if daily_food >= self.daily_food_requirement:
                        break
            
            if daily_food < self.daily_food_requirement:
                print(f"    Day ended with insufficient food: {daily_food:.1f}kg / {self.daily_food_requirement}kg")
                
            total_food_acquired += daily_food
            
        avg_daily_food = total_food_acquired / days
        hunt_success_rate = successful_hunts / total_hunts if total_hunts > 0 else 0
        
        return avg_daily_food, hunt_success_rate, {'zones': zone_stats, 'prey': prey_stats}

# Create example data
prey_types = [
    Prey("Gazelle", speed=80, weight=50, herd_size=20, daily_arrival_rate=2.5),
    Prey("Wildebeest", speed=60, weight=200, herd_size=50, daily_arrival_rate=0.8),
    Prey("Zebra", speed=65, weight=350, herd_size=30, daily_arrival_rate=1.2)
]

hunting_zones = [
    HuntingZone("Riverside", distance_from_pride=800, water_proximity=100, cover_density=0.7),
    HuntingZone("Savanna", distance_from_pride=2000, water_proximity=500, cover_density=0.3),
    HuntingZone("Woodland", distance_from_pride=1500, water_proximity=300, cover_density=0.8)
]

# Run simulation
pride_manager = PrideResourceManager(hunting_zones, prey_types)
avg_food, success_rate = pride_manager.simulate_daily_outcomes(days=30)

# Run simulation
print("\nStarting Pride Resource Management Simulation...")
print(f"Pride size: 6 lions")
print(f"Daily food requirement: 30kg")
print("\nHunting Zones:")
for zone in hunting_zones:
    print(f"- {zone.name}: {zone.distance_from_pride}m from pride, {zone.water_proximity}m from water, {zone.cover_density:.1%} cover density")
print("\nPrey Types:")
for prey in prey_types:
    print(f"- {prey.species}: {prey.speed}m/s speed, {prey.weight}kg weight, herds of {prey.herd_size}, {prey.daily_arrival_rate} herds/day average")

pride_manager = PrideResourceManager(hunting_zones, prey_types)
avg_food, success_rate, stats = pride_manager.simulate_daily_outcomes(days=7)  # Reduced to 7 days for clearer output

print("\nSimulation Results:")
print(f"Average daily food acquired: {avg_food:.1f}kg")
print(f"Overall hunt success rate: {success_rate:.1%}")

print("\nSuccess rates by hunting zone:")
for zone_name, zone_data in stats['zones'].items():
    success_rate = zone_data['successes'] / zone_data['attempts'] if zone_data['attempts'] > 0 else 0
    print(f"- {zone_name}: {success_rate:.1%} ({zone_data['successes']}/{zone_data['attempts']} successful hunts)")

print("\nSuccess rates by prey type:")
for prey_name, prey_data in stats['prey'].items():
    success_rate = prey_data['successes'] / prey_data['attempts'] if prey_data['attempts'] > 0 else 0
    print(f"- {prey_name}: {success_rate:.1%} ({prey_data['successes']}/{prey_data['attempts']} successful hunts)")


Day 1:
  2 Gazelle herds spotted in Riverside
  1 Wildebeest herds spotted in Riverside
  3 Zebra herds spotted in Riverside
  3 Gazelle herds spotted in Savanna
  4 Zebra herds spotted in Savanna
  5 Gazelle herds spotted in Woodland
  1 Wildebeest herds spotted in Woodland
  1 Zebra herds spotted in Woodland
    Day ended with insufficient food: 0.0kg / 30kg

Day 2:
  2 Gazelle herds spotted in Riverside
    Successful hunt! Caught Gazelle (+50kg)
    Daily requirement met: 50.0kg
  2 Gazelle herds spotted in Savanna
  3 Gazelle herds spotted in Woodland

Day 3:
  5 Gazelle herds spotted in Riverside
    Successful hunt! Caught Gazelle (+50kg)
    Daily requirement met: 50.0kg
  3 Gazelle herds spotted in Savanna
  1 Gazelle herds spotted in Woodland

Day 4:
  2 Gazelle herds spotted in Riverside
  3 Zebra herds spotted in Riverside
  6 Gazelle herds spotted in Savanna
  2 Wildebeest herds spotted in Savanna
    Successful hunt! Caught Wildebeest (+200kg)
    Daily requirement met: 

ValueError: too many values to unpack (expected 2)